# 05 Tooling | 01 Semantic Kernel | 01 Plugin

## Azure Environment

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/01_Environment.ipynb) to setup the necessary demo environment.

This notebooks highlights how Plugins work. 

## Step 1: Create Semantic Kernel

MS Semantic Kernel is a tool or library developed by Microsoft to abstract away communication with LLMs and other services like vector DBs. The nuget package Microsoft.SemanticKernel provides developers with access to this powerful tool, allowing them to integrate advanced language processing capabilities into their applications effortlessly.


In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.48.0"
#r "nuget: DotNetEnv, 3.1.1"

using System.IO;
using System.ComponentModel;
using DotNetEnv;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;

//configuration file is created during environment creation
static string _configurationFile = @"../../Configuration/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";
string assetsFolder = Environment.GetEnvironmentVariable("WS_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";

Console.WriteLine($"Configuration loaded...");


Installed Packages DotNetEnv, 3.1.1 Microsoft.SemanticKernel, 1.48.0

Configuration loaded...


## Step 2: Define Plugin

The code cell defines a so called plugin. A plugin is functionality (both native or semantic) which can be executed by Semantic Kernel. In this sample a summarization plugin is defined.



In [2]:
class CodeLibrary
{
    [KernelFunction("get_sport_event_winner")]
    //Description is just necessary if function name is not self-explanatory
    [Description("Get the winner of a sport event. The event is identified by the sport event name and the year.")]
    public string GetSportEventWinner(string sportEventName = "", string sportEventYear = "")
    {
        // Implement the logic to get the winner of the sport event.
        return "Kansas City Chiefs";
    }

    [KernelFunction("get_sport_event_score")]
    //Description is just necessary if function name is cryptic and not self-explanatory
    [Description("Get the score of a specific sport event. The event is identified by the sport event name and the year.")]
    public string GetSportEventScore(string sportEventName = "", string sportEventYear = "")
    {
        // Implement the logic to get the result of the sport event.
        return "25-22";
    }

    [KernelFunction("GetData")]
    [Description("Get the score and winner of a specific sport event.")]
    
    public string GetData(){
        return "Kansas City Chiefs won 25-22";
    }

}

Console.WriteLine($"Code library defined...");

Code library defined...


## Step 3: Define Kernel instance

Definition of a Kernel instance with the previously defined `CodeLibrary` class registered as a PlugIn. 



In [3]:
IKernelBuilder kernelBuilder = Kernel.CreateBuilder();

kernelBuilder.AddAzureOpenAIChatCompletion(
    apiKey: oAiApiKey, 
    endpoint: oAiEndpoint, 
    deploymentName: chatCompletionDeploymentName
);

kernelBuilder.Plugins.AddFromType<CodeLibrary>("CodeLibrary");
Kernel kernel = kernelBuilder.Build(); 

IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();


Console.WriteLine($"Kernel created...");

Kernel created...


## Step 4: Execute Prompt

In [4]:
string prompt = "Who won the Super Bowl 2024. Return team name and score.";
ChatHistory chatHistory = new ChatHistory();
chatHistory.AddUserMessage(prompt);

PromptExecutionSettings  promptExecutionSettings = new PromptExecutionSettings();
promptExecutionSettings.FunctionChoiceBehavior = FunctionChoiceBehavior.Auto();

ChatMessageContent chatMessageContent = await chatCompletionService.GetChatMessageContentAsync(
    chatHistory: chatHistory, 
    executionSettings: promptExecutionSettings,
    kernel: kernel
);

Console.WriteLine($"{chatMessageContent.Content}");

The winner of the Super Bowl 2024 was the Kansas City Chiefs with a score of 25-22.
